# Inference of Trained Retinanet model 
### Import Libraries

In [1]:
# show images inline
%matplotlib inline

# automatically reload modules when they have changed
%load_ext autoreload
%autoreload 2

# import keras
import keras

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color
from keras_retinanet.utils.gpu import setup_gpu

# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
from numpy import genfromtxt
import time
import json
import copy

# use this to change which GPU to use
gpu = 0

# set the modified tf session as backend in keras
#setup_gpu(gpu)

Using TensorFlow backend.


In [2]:
file_location = 'list_of_img_in_val_set4.csv'
place_to_store_results = 'Evaluations/'
path_img_folder = '../../../03 Data/Simple Dataset/'
accept_BB_threshold = 0.7

def convert_rect_to_point(rect):
    x = rect[0]
    y = rect[1]
    w = rect[2]
    h = rect[3]
    x1 = x
    y1 = y
    x2 = x + w
    y2 = y + h
    
    return (x1, y1, x2, y2)

def read_annotations_from_json(img_name):
    with open(path_img_folder + img_name.strip('.png') + '.json') as json_file:
        data = json.load(json_file)
        shapes_list = data['shapes']
        annotation_list = []
        for annotation in shapes_list: 
            rect = cv2.boundingRect(np.float32(annotation['points']))
            annotation_list.append(convert_rect_to_point(rect))
    return annotation_list; 


def max_val(val1, val2):
    if val1 > val2:
        return val1
    else:
        return val2
    
def min_val(val1, val2):
    if val1 > val2:
        return val2
    else:
        return val1
    
    
def filter_bounding_boxes(bounding_boxes, scores):
    list_of_BB = []
    for box, score in zip(bounding_boxes, scores):
        if score < accept_BB_threshold: 
            break
        list_of_BB.append(box)
    return list_of_BB


def area(BB):
    width = abs(BB[0] - BB[2])
    height = abs(BB[1] - BB[3])
    return height * width

def intersection(BB1, BB2):
    # find coordiantes of intersection rectangle 
    (x1, y1, x2, y2) = BB1
    (x3, y3, x4, y4) = BB2
    if x1 > x4 or x3 > x2 or y3 > y2 or y1 > y4:
        return 0
    x5 = max_val(x1, x3);
    y5 = max_val(y1, y3);
    x6 = min_val(x2, x4);
    y6 = min_val(y2, y4);  
    BB = (x5, y5, x6, y6)
    # TODO: check if no intersection exists
    return area(BB)
    
def union(BB1, BB2): 
    return area(BB1) + area(BB2) - intersection(BB1, BB2)

def evaluate_predictions(annotations, preds, IOU_thres):
    predictions = copy.deepcopy(preds)
    false_neg = 0
    for annotation in annotations: 
        #annotation = convert_rect_to_point(anno)
        max_IOU = 0
        for i in range(len(predictions)): 
            #print('Annotation: ', annotation)
            #print('pred candidate: ', predictions[i])
            if intersection(annotation, predictions[i]) > 0: #convert_rect_to_point only for tests
                IOU = intersection(annotation, predictions[i])/union(annotation, predictions[i])
                #print(IOU)  
                if(IOU > max_IOU):
                    max_IOU_index = i
                    max_IOU = IOU
        if max_IOU > IOU_thres:
            # remove prediction from list
            predictions.pop(max_IOU_index)
        else:
            false_neg = false_neg + 1
    
    false_pos = len(predictions)
    true_pos = len(preds) - len(predictions)
    return (false_neg, false_pos, true_pos)        

### Import Data and Model 

In [3]:
# import data
file = open(file_location)
file_paths = list(file)  

print(file_paths)

# import model
model_path = os.path.join('..','..','Create-CSV-dataset', 'snapshots', 'resnet50_csv_25.h5');

model = models.load_model(model_path, backbone_name='resnet50');

# If model is not converted to inference model, use line below: 
model = models.convert_model(model)


labels_to_names = {0: 'Brick'};

#labels_to_names = {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microwave', 69: 'oven', 70: 'toaster', 71: 'sink', 72: 'refrigerator', 73: 'book', 74: 'clock', 75: 'vase', 76: 'scissors', 77: 'teddy bear', 78: 'hair drier', 79: 'toothbrush'}

['colorIMG_75.png\n', 'colorIMG_32.png\n', 'colorIMG_114.png\n', 'colorIMG_95.png\n', 'colorIMG_43.png\n', 'colorIMG_67.png\n', 'colorIMG_24.png\n', 'colorIMG_12.png\n', 'colorIMG_159.png\n', 'colorIMG_51.png\n', 'colorIMG_103.png\n', 'colorIMG_96.png\n', 'colorIMG_50.png\n', 'colorIMG_179.png\n', 'colorIMG_94.png\n', 'colorIMG_88.png\n', 'colorIMG_77.png\n', 'colorIMG_108.png\n', 'colorIMG_37.png\n', 'colorIMG_152.png\n', 'colorIMG_9.png\n', 'colorIMG_45.png\n', 'colorIMG_99.png\n', 'colorIMG_111.png\n', 'colorIMG_201.png\n', 'colorIMG_78.png\n', 'colorIMG_47.png\n', 'colorIMG_15.png\n', 'colorIMG_46.png\n', 'colorIMG_183.png\n', 'colorIMG_62.png\n', 'colorIMG_136.png\n', 'colorIMG_116.png\n', 'colorIMG_147.png\n', 'colorIMG_53.png\n', 'colorIMG_107.png\n', 'colorIMG_132.png\n', 'colorIMG_44.png\n', 'colorIMG_190.png\n', 'colorIMG_55.png\n']
tracking <tf.Variable 'Variable:0' shape=(9, 4) dtype=float32, numpy=
array([[-22.627417, -11.313708,  22.627417,  11.313708],
       [-28.50876 

### Infer Images

In [9]:
IOU_threshold = 0.5
total_false_positive = 0
total_true_positive = 0
total_false_negative = 0

for path in file_paths:
    image = read_image_bgr(path_img_folder + path.strip('\n'))

    # create copy to draw on 
    draw = image.copy()
    #draw = cv2.cvtColor(draw, cv2.COLhttp://cocodataset.org/#detection-evalOR_BGR2RGB)
    
    # preprocess image 
    # TODOD: check if preprocess_image convert the image to RGB format
    image = preprocess_image(image)
    image, scale = resize_image(image)
    
    # process image 
    prediction = model.predict_on_batch(np.expand_dims(image, axis=0))
    print(prediction)

[array([[[727.7565 , 439.76175, 909.0072 , 516.0489 ],
        [729.7914 , 483.97034, 903.4134 , 559.85645],
        [721.8939 , 530.19293, 900.0026 , 603.70135],
        ...,
        [ -1.     ,  -1.     ,  -1.     ,  -1.     ],
        [ -1.     ,  -1.     ,  -1.     ,  -1.     ],
        [ -1.     ,  -1.     ,  -1.     ,  -1.     ]]], dtype=float32), array([[ 0.99999774,  0.999997  ,  0.9999943 ,  0.99999213,  0.9999894 ,
         0.99998546,  0.9999827 ,  0.999977  ,  0.99996483,  0.99995923,
         0.99995494,  0.9999529 ,  0.99995065,  0.99988806,  0.99981135,
         0.9996457 ,  0.9036397 ,  0.56252104,  0.12732776,  0.06889299,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , 

[array([[[542.5152 , 360.8421 , 714.8587 , 424.9785 ],
        [546.2487 , 322.0902 , 717.97797, 381.79776],
        [528.9802 , 532.0395 , 699.6751 , 591.2152 ],
        ...,
        [ -1.     ,  -1.     ,  -1.     ,  -1.     ],
        [ -1.     ,  -1.     ,  -1.     ,  -1.     ],
        [ -1.     ,  -1.     ,  -1.     ,  -1.     ]]], dtype=float32), array([[ 0.99999404,  0.99999225,  0.99998546,  0.9999844 ,  0.9999832 ,
         0.9999821 ,  0.9999789 ,  0.9999714 ,  0.99992335,  0.9999138 ,
         0.9998092 ,  0.9997949 ,  0.9996921 ,  0.9996524 ,  0.99964345,
         0.998944  ,  0.65118265,  0.05916829, -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , 

[array([[[545.5551 , 426.44525, 718.9304 , 484.61826],
        [548.2986 , 383.43542, 721.0928 , 442.22623],
        [544.5628 , 508.84073, 717.91974, 568.6478 ],
        ...,
        [ -1.     ,  -1.     ,  -1.     ,  -1.     ],
        [ -1.     ,  -1.     ,  -1.     ,  -1.     ],
        [ -1.     ,  -1.     ,  -1.     ,  -1.     ]]], dtype=float32), array([[ 0.9999944 ,  0.9999912 ,  0.99999   ,  0.99998736,  0.99998724,
         0.99996245,  0.9999554 ,  0.9999496 ,  0.9999366 ,  0.99993086,
         0.9998838 ,  0.9997975 ,  0.9997018 ,  0.9995309 ,  0.99926585,
         0.99891555,  0.2560185 ,  0.07059792, -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , 

[array([[[711.1444 , 284.8807 , 887.55835, 348.073  ],
        [704.91187, 371.94742, 880.2383 , 430.69775],
        [706.3452 , 332.82266, 879.60626, 387.0581 ],
        ...,
        [ -1.     ,  -1.     ,  -1.     ,  -1.     ],
        [ -1.     ,  -1.     ,  -1.     ,  -1.     ],
        [ -1.     ,  -1.     ,  -1.     ,  -1.     ]]], dtype=float32), array([[ 0.99999416,  0.99999213,  0.9999893 ,  0.99998367,  0.9999827 ,
         0.9999696 ,  0.999969  ,  0.99994564,  0.9999454 ,  0.9999418 ,
         0.9999294 ,  0.9999014 ,  0.99989164,  0.9998815 ,  0.999853  ,
         0.9992749 ,  0.5535323 ,  0.12430978, -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , 

[array([[[694.3846 , 291.564  , 827.86615, 442.7309 ],
        [655.10693, 272.7854 , 795.49365, 423.98785],
        [790.2023 , 365.8812 , 924.91797, 523.02966],
        ...,
        [ -1.     ,  -1.     ,  -1.     ,  -1.     ],
        [ -1.     ,  -1.     ,  -1.     ,  -1.     ],
        [ -1.     ,  -1.     ,  -1.     ,  -1.     ]]], dtype=float32), array([[ 0.9999844 ,  0.9999751 ,  0.9999528 ,  0.99994314,  0.999931  ,
         0.99987483,  0.9998658 ,  0.9998505 ,  0.9998387 ,  0.9998016 ,
         0.9997925 ,  0.9997733 ,  0.9997217 ,  0.99963176,  0.99956316,
         0.16338919,  0.1506584 ,  0.11693098,  0.1071981 ,  0.10512316,
         0.10071392,  0.0635663 ,  0.05596231,  0.05103113, -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , 

...

[array([[[703.7584 , 481.25156, 879.6604 , 552.9333 ],
        [549.9147 , 376.57907, 723.8292 , 441.8624 ],
        [716.96185, 399.40338, 892.31854, 471.223  ],
        ...,
        [ -1.     ,  -1.     ,  -1.     ,  -1.     ],
        [ -1.     ,  -1.     ,  -1.     ,  -1.     ],
        [ -1.     ,  -1.     ,  -1.     ,  -1.     ]]], dtype=float32), array([[ 0.99999857,  0.9999951 ,  0.9999949 ,  0.9999949 ,  0.9999927 ,
         0.99999225,  0.9999914 ,  0.9999912 ,  0.9999902 ,  0.9999844 ,
         0.9999821 ,  0.9999752 ,  0.9999734 ,  0.99996483,  0.99994755,
         0.99994016,  0.8647026 ,  0.16519944,  0.10242575,  0.09521899,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , 

[array([[[698.7375 , 487.051  , 873.86615, 548.5093 ],
        [695.1201 , 526.6598 , 869.96747, 606.65076],
        [719.8862 , 239.61902, 890.00085, 288.54678],
        ...,
        [ -1.     ,  -1.     ,  -1.     ,  -1.     ],
        [ -1.     ,  -1.     ,  -1.     ,  -1.     ],
        [ -1.     ,  -1.     ,  -1.     ,  -1.     ]]], dtype=float32), array([[ 0.9999913 ,  0.99999   ,  0.9999821 ,  0.9998869 ,  0.9998766 ,
         0.9998617 ,  0.9998586 ,  0.9997229 ,  0.999703  ,  0.9994165 ,
         0.9990085 ,  0.99852246,  0.9980715 ,  0.9973744 ,  0.9973086 ,
         0.99618393,  0.66415673,  0.44294062,  0.05393273,  0.05386148,
         0.05183094, -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , 

[array([[[708.0248 , 562.60925, 884.3883 , 634.44025],
        [560.25226, 249.74239, 731.6281 , 305.29462],
        [711.4239 , 520.33435, 886.8465 , 580.57947],
        ...,
        [ -1.     ,  -1.     ,  -1.     ,  -1.     ],
        [ -1.     ,  -1.     ,  -1.     ,  -1.     ],
        [ -1.     ,  -1.     ,  -1.     ,  -1.     ]]], dtype=float32), array([[ 0.99999774,  0.9999927 ,  0.9999864 ,  0.99997926,  0.999977  ,
         0.999967  ,  0.9999597 ,  0.999959  ,  0.99995625,  0.99994576,
         0.9999454 ,  0.9999336 ,  0.9998435 ,  0.9998191 ,  0.99967897,
         0.99961877,  0.87624335,  0.45644438,  0.24089347,  0.06118128,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , 

[array([[[592.91   , 236.91719, 638.9861 , 402.95563],
        [543.5217 , 241.87758, 591.7259 , 407.47296],
        [675.7689 , 228.76443, 720.9415 , 396.24042],
        ...,
        [ -1.     ,  -1.     ,  -1.     ,  -1.     ],
        [ -1.     ,  -1.     ,  -1.     ,  -1.     ],
        [ -1.     ,  -1.     ,  -1.     ,  -1.     ]]], dtype=float32), array([[ 0.99981254,  0.99973375,  0.99954396,  0.9994211 ,  0.99899966,
         0.99894136,  0.99831855,  0.9982804 ,  0.99780124,  0.9977239 ,
         0.99760497,  0.99748385,  0.9974511 ,  0.9968497 ,  0.9938326 ,
         0.9880822 ,  0.85294366,  0.78019536,  0.5084523 ,  0.32828376,
         0.3159096 ,  0.19032915,  0.17052425,  0.12237751,  0.07183766,
         0.05979644,  0.05612304, -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , 

[array([[[528.67474, 507.24216, 699.6718 , 595.18463],
        [508.83737, 426.20383, 681.6229 , 514.69275],
        [539.75867, 590.9813 , 716.50995, 675.9889 ],
        ...,
        [ -1.     ,  -1.     ,  -1.     ,  -1.     ],
        [ -1.     ,  -1.     ,  -1.     ,  -1.     ],
        [ -1.     ,  -1.     ,  -1.     ,  -1.     ]]], dtype=float32), array([[ 0.9999949 ,  0.9999832 ,  0.9999813 ,  0.9999788 ,  0.9999691 ,
         0.99996436,  0.99996376,  0.99996114,  0.9999529 ,  0.9999441 ,
         0.99992514,  0.99991596,  0.9999093 ,  0.9998827 ,  0.9998134 ,
         0.9996916 ,  0.10722154,  0.09457159,  0.06867316,  0.0603754 ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , 

In [4]:
IOU_threshold = 0.5
total_false_positive = 0
total_true_positive = 0
total_false_negative = 0

for path in file_paths:
    image = read_image_bgr(path_img_folder + path.strip('\n'))

    # create copy to draw on 
    draw = image.copy()
    #draw = cv2.cvtColor(draw, cv2.COLhttp://cocodataset.org/#detection-evalOR_BGR2RGB)
    
    # preprocess image 
    # TODOD: check if preprocess_image convert the image to RGB format
    image = preprocess_image(image)
    image, scale = resize_image(image)
    
    # process image 
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
    
    # correct for image scale
    boxes /= scale
    
    # print detections
    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        # scores are sorted so we can break
        if score < accept_BB_threshold:
            break

        color = label_color(label)

        b = box.astype(int)
        draw_box(draw, b, color=color)

        caption = "{} {:.3f}".format(labels_to_names[label], score)
        draw_caption(draw, b, caption)
    
    # load json file containing annotations
    annotations = read_annotations_from_json(path.strip('\n'))
    
    # remove bounding boxes with a classification score below accept_BB_threshold. 
    predictions = filter_bounding_boxes(boxes[0], scores[0])
    
    #calculate intersection and union of two bounding boxes
    (false_neg, false_pos, true_pos) = evaluate_predictions(annotations, predictions, IOU_threshold)
    #results = evaluate_predictions(annotations, annotations)
    total_false_negative = total_false_negative + false_neg
    total_false_positive = total_false_positive + false_pos
    total_true_positive  = total_true_positive  + true_pos
    
    # store image
    #cv2.imwrite(place_to_store_results + path.strip('\n'), draw)

# Calculate precision, recall and F1 score 
precision = total_true_positive / (total_true_positive + total_false_positive)
recall = total_true_positive / (total_true_positive + total_false_negative)
f1 = 2 * (precision * recall) / (precision + recall) 

# print result
print('Precision: ', precision)
print('Recall: ', recall)
print('f1: ', f1)

Precision:  0.962178517397882
Recall:  0.99375
f1:  0.9777094542659492
